In [49]:
import paramiko
from scp import SCPClient
import shutil
import os
from pathlib import Path
import time

# ==== TPA Simulation Remote Execution System ====
print("=== TPA Simulation Remote Launcher ===")
print("Preparing files and submitting jobs to remote server...")
print()

# Configuration
server = "tcad01.hep.manchester.ac.uk"
username = "lihuazhen"
local_satellite_dir = "Garfield_satellite"
remote_path = "/tmp/"
bash_script_name = "run_tpa_simulation.sh"

# Files to copy from Garfield_workplace
source_dir = "../Garfield_workplace"
files_to_copy = [
    "Diamond_4p.C",
    "LUT.csv", 
    "run_tpa_simulation.sh",
    "CMakeLists.txt"
]


=== TPA Simulation Remote Launcher ===
Preparing files and submitting jobs to remote server...



In [50]:
# ==== Step 1: Prepare local satellite directory ====
print("Step 1: Preparing local satellite directory...")

# Clear the satellite directory
if os.path.exists(local_satellite_dir):
    print(f"  → Clearing existing {local_satellite_dir}/")
    shutil.rmtree(local_satellite_dir)

# Create fresh satellite directory
os.makedirs(local_satellite_dir, exist_ok=True)
print(f"  → Created fresh {local_satellite_dir}/")

# Copy required files from Garfield_workplace
print(f"  → Copying files from {source_dir}/")
copied_files = []
for file_name in files_to_copy:
    source_file = os.path.join(source_dir, file_name)
    dest_file = os.path.join(local_satellite_dir, file_name)
    
    if os.path.exists(source_file):
        shutil.copy2(source_file, dest_file)
        file_size = os.path.getsize(dest_file)
        print(f"    ✓ {file_name} ({file_size:,} bytes)")
        copied_files.append(file_name)
    else:
        print(f"    ✗ {file_name} (not found)")

print(f"  → Successfully copied {len(copied_files)}/{len(files_to_copy)} files")
print()


Step 1: Preparing local satellite directory...
  → Clearing existing Garfield_satellite/
  → Created fresh Garfield_satellite/
  → Copying files from ../Garfield_workplace/
    ✓ Diamond_4p.C (11,080 bytes)
    ✓ LUT.csv (3,913,068 bytes)
    ✓ run_tpa_simulation.sh (7,059 bytes)
    ✓ CMakeLists.txt (472 bytes)
  → Successfully copied 4/4 files



In [51]:
# ==== Step 2: Upload to remote server ====
print("Step 2: Uploading to remote server...")

try:
    # Establish SSH connection
    print(f"  → Connecting to {username}@{server}")
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(server, username=username)
    print("  ✓ SSH connection established")
    
    # Upload satellite directory
    print(f"  → Uploading {local_satellite_dir}/ to {remote_path}")
    with SCPClient(ssh.get_transport()) as scp:
        scp.put(local_satellite_dir, remote_path, recursive=True)
    print("  ✓ Upload completed")
    
    # Make bash script executable
    remote_script_path = f"{remote_path}Garfield_satellite/{bash_script_name}"
    ssh.exec_command(f"chmod +x {remote_script_path}")
    print(f"  ✓ Made {bash_script_name} executable")
    
except Exception as e:
    print(f"  ✗ Error during upload: {e}")
    raise

print()


Step 2: Uploading to remote server...
  → Connecting to lihuazhen@tcad01.hep.manchester.ac.uk
  ✓ SSH connection established
  → Uploading Garfield_satellite/ to /tmp/
  ✓ Upload completed
  ✓ Made run_tpa_simulation.sh executable



In [52]:
# ==== Step 3: Execute simulation on remote server ====
print("Step 3: Executing TPA simulation on remote server...")

try:
    # Execute the bash script
    remote_dir = f"{remote_path}Garfield_satellite"
    command = f"cd {remote_dir} && ./{bash_script_name}"
    
    print(f"  → Executing: {command}")
    print("  → This may take several minutes to hours...")
    print("  → You can monitor progress in real-time below:")
    print("  " + "="*60)
    
    # Execute command and stream output in real-time
    transport = ssh.get_transport()
    channel = transport.open_session()
    channel.exec_command(command)
    
    # Stream output in real-time
    while True:
        if channel.recv_ready():
            data = channel.recv(1024).decode('utf-8')
            if data:
                print(data, end='')
        
        if channel.recv_stderr_ready():
            error_data = channel.recv_stderr(1024).decode('utf-8')
            if error_data:
                print(f"[STDERR] {error_data}", end='')
        
        if channel.exit_status_ready():
            break
        time.sleep(0.1)
    
    exit_status = channel.recv_exit_status()
    channel.close()
    
    print("  " + "="*60)
    if exit_status == 0:
        print("  ✓ Simulation completed successfully!")
    else:
        print(f"  ✗ Simulation failed with exit status: {exit_status}")
        
except Exception as e:
    print(f"  ✗ Error during execution: {e}")
    raise

print()


Step 3: Executing TPA simulation on remote server...
  → Executing: cd /tmp/Garfield_satellite && ./run_tpa_simulation.sh
  → This may take several minutes to hours...
  → You can monitor progress in real-time below:
=== TPA Simulation Runner ===
Compilation and batch job submission script

Step 1: Compiling Diamond_4p...
Cleaning previous build...
  → Removed existing build directory
  → Created fresh build directory
Running cmake and make...
-- The C compiler identification is GNU 8.5.0
-- The CXX compiler identification is GNU 8.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Vdt: /opt/root/include (found version "0

KeyboardInterrupt: 

In [ ]:
# ==== Step 4: Download results ====
print("Step 4: Downloading results from remote server...")

try:
    # Create local results directory under Results/
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    results_base_dir = "../Results"
    # Ensure Results directory exists
    os.makedirs(results_base_dir, exist_ok=True)
    local_results_dir = os.path.join(results_base_dir, f"TPA_results_{timestamp}")
    os.makedirs(local_results_dir, exist_ok=True)
    print(f"  → Created local results directory: {local_results_dir}/")
    
    # Check what result directories exist on remote server
    stdin, stdout, stderr = ssh.exec_command(f"ls -la {remote_dir}/TPA_results_*/ 2>/dev/null | head -20")
    remote_dirs = stdout.read().decode().strip()
    
    if remote_dirs:
        print("  → Found result directories on remote server:")
        print("    " + remote_dirs.replace('\n', '\n    '))
        
        # Get the most recent results directory
        stdin, stdout, stderr = ssh.exec_command(f"ls -1 {remote_dir}/TPA_results_*/ | tail -1")
        latest_dir = stdout.read().decode().strip()
        
        if latest_dir:
            remote_results_path = f"{remote_dir}/{latest_dir.rstrip('/')}"
            print(f"  → Downloading from: {remote_results_path}")
            
            # Download results
            with SCPClient(ssh.get_transport()) as scp:
                scp.get(remote_results_path, local_results_dir, recursive=True)
            
            # Count downloaded files
            downloaded_files = []
            for root, dirs, files in os.walk(local_results_dir):
                for file in files:
                    if file.endswith('.txt') or file.endswith('.log'):
                        downloaded_files.append(file)
            
            print(f"  ✓ Downloaded {len(downloaded_files)} result files")
            print(f"  → Results saved to: {local_results_dir}/")
            
            # Show summary of downloaded files
            tpa_files = [f for f in downloaded_files if f.startswith('TPA_simulation_')]
            log_files = [f for f in downloaded_files if f.endswith('.log')]
            
            print(f"    - TPA result files: {len(tpa_files)}")
            print(f"    - Log files: {len(log_files)}")
        else:
            print("  ✗ No result directories found")
    else:
        print("  ⚠️  No result directories found on remote server")
        print("  → Check if simulation completed successfully")
        
except Exception as e:
    print(f"  ✗ Error during download: {e}")
    # Don't raise here, as we still want to cleanup

print()


Step 4: Downloading results from remote server...
  → Created local results directory: ../Results/TPA_results_20250611_175144/
  → Found result directories on remote server:
    /tmp/Garfield_satellite/TPA_results_20250611_163545/:
    total 8
    drwxr-xr-x 2 lihuazhen atlas 4096 Jun 11 16:35 .
    drwxr-xr-x 8 lihuazhen atlas 4096 Jun 11 16:51 ..
    
    /tmp/Garfield_satellite/TPA_results_20250611_163937/:
    total 8
    drwxr-xr-x 2 lihuazhen atlas 4096 Jun 11 16:39 .
    drwxr-xr-x 8 lihuazhen atlas 4096 Jun 11 16:51 ..
    
    /tmp/Garfield_satellite/TPA_results_20250611_164325/:
    total 8
    drwxr-xr-x 2 lihuazhen atlas 4096 Jun 11 16:43 .
    drwxr-xr-x 8 lihuazhen atlas 4096 Jun 11 16:51 ..
    
    /tmp/Garfield_satellite/TPA_results_20250611_164608/:
    total 204
    drwxr-xr-x 2 lihuazhen atlas 4096 Jun 11 16:46 .
    drwxr-xr-x 8 lihuazhen atlas 4096 Jun 11 16:51 ..
    -rw-r--r-- 1 lihuazhen atlas 1201 Jun 11 16:46 job_x12.500000_y12.500000.log
  → Downloading from

In [48]:
# ==== Step 5: Cleanup and Summary ====
print("Step 5: Cleanup and summary...")

try:
    # Close SSH connection
    ssh.close()
    print("  ✓ SSH connection closed")
    
    # Optional: Clean up remote files (uncomment if desired)
    # print("  → Cleaning up remote files...")
    # ssh_cleanup = paramiko.SSHClient()
    # ssh_cleanup.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    # ssh_cleanup.connect(server, username=username)
    # ssh_cleanup.exec_command(f"rm -rf {remote_path}Garfield_satellite")
    # ssh_cleanup.close()
    # print("  ✓ Remote files cleaned up")
    
except Exception as e:
    print(f"  ⚠️  Error during cleanup: {e}")

print()
print("="*60)
print("🎉 TPA Simulation Remote Execution Complete!")
print("="*60)

# Final summary
if 'local_results_dir' in locals() and os.path.exists(local_results_dir):
    total_size = sum(os.path.getsize(os.path.join(root, file)) 
                    for root, dirs, files in os.walk(local_results_dir) 
                    for file in files)
    print(f"📁 Results directory: {local_results_dir}/")
    print(f"📊 Total size: {total_size:,} bytes")
    print(f"🔗 Remote server: {username}@{server}")
    print(f"📍 Remote path: {remote_path}Garfield_satellite/")
else:
    print("⚠️  No results were downloaded")

print("\n💡 Next steps:")
print("   1. Check the downloaded result files")
print("   2. Analyze the TPA simulation data") 
print("   3. Process the results in your analysis notebooks")

print(f"\n🕒 Execution completed at: {time.strftime('%Y-%m-%d %H:%M:%S')}")


Step 5: Cleanup and summary...
  ✓ SSH connection closed

🎉 TPA Simulation Remote Execution Complete!
📁 Results directory: ../Results/TPA_results_20250611_175144/
📊 Total size: 0 bytes
🔗 Remote server: lihuazhen@tcad01.hep.manchester.ac.uk
📍 Remote path: /tmp/Garfield_satellite/

💡 Next steps:
   1. Check the downloaded result files
   2. Analyze the TPA simulation data
   3. Process the results in your analysis notebooks

🕒 Execution completed at: 2025-06-11 17:51:50
